In [1]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.10 ; cuda:  1.10.0


In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("road_defect_detection_train", {}, "instances_default.json", "training_images")
register_coco_instances("road_defect_detection_val", {}, "cropped_images/09_labelled/annotations/instances_default.json", "cropped_images/09_labelled/images")

my_dataset_train_metadata = MetadataCatalog.get("road_defect_detection_train")
dataset_dicts = DatasetCatalog.get("road_defect_detection_train")

[05/28 08:42:12 d2.data.datasets.coco]: Loaded 269 images in COCO format from instances_default.json


In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.WEIGHTS = 'model_final.pth'
cfg.DATASETS.TRAIN = ("road_defect_detection_train",)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [6]:
from detectron2.utils.visualizer import ColorMode
# import matplotlib.pyplot as plt
test_path = "cropped_images/09_labelled/images"

for filename in os.listdir(test_path):
    f = os.path.join(test_path, filename)
    print(f)
    im = cv2.imread(f)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    out_filename = filename[:-4] + "_out.png"
    cv2.imwrite(out_filename, v.get_image()[:, :, ::-1])

cropped_images/09_labelled/images\tile_09_v2_0_0.png
cropped_images/09_labelled/images\tile_09_v2_0_10200.png
cropped_images/09_labelled/images\tile_09_v2_0_11475.png
cropped_images/09_labelled/images\tile_09_v2_0_1275.png
cropped_images/09_labelled/images\tile_09_v2_0_12750.png
cropped_images/09_labelled/images\tile_09_v2_0_14025.png
cropped_images/09_labelled/images\tile_09_v2_0_2550.png
cropped_images/09_labelled/images\tile_09_v2_0_3825.png
cropped_images/09_labelled/images\tile_09_v2_0_5100.png
cropped_images/09_labelled/images\tile_09_v2_0_6375.png
cropped_images/09_labelled/images\tile_09_v2_0_7650.png
cropped_images/09_labelled/images\tile_09_v2_0_8925.png
cropped_images/09_labelled/images\tile_09_v2_850_0.png
cropped_images/09_labelled/images\tile_09_v2_850_10200.png
cropped_images/09_labelled/images\tile_09_v2_850_11475.png
cropped_images/09_labelled/images\tile_09_v2_850_1275.png
cropped_images/09_labelled/images\tile_09_v2_850_12750.png
cropped_images/09_labelled/images\til

In [5]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("road_defect_detection_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "road_defect_detection_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

[05/28 08:42:59 d2.data.datasets.coco]: Loaded 24 images in COCO format from cropped_images/09_labelled/annotations/instances_default.json
[05/28 08:42:59 d2.data.build]: Distribution of instances among all 8 categories:
|  category  | #instances   |   category    | #instances   |   category    | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|   Cracks   | 31           |   Potholes    | 33           |    Patches    | 0            |
|  Sidewalk  | 0            | White Road .. | 0            | Double Yell.. | 0            |
|    Car     | 0            | Alligator C.. | 4            |               |              |
|   total    | 68           |               |              |               |              |
[05/28 08:42:59 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/28 08:42:59 d2.data.common]: Serializing 24 

C:\Users\luope\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/28 08:43:15 d2.evaluation.evaluator]: Inference done 11/24. Dataloading: 0.0005 s/iter. Inference: 0.2280 s/iter. Eval: 0.0071 s/iter. Total: 0.2355 s/iter. ETA=0:00:03
[05/28 08:43:18 d2.evaluation.evaluator]: Total inference time: 0:00:05.086842 (0.267729 s / iter per device, on 1 devices)
[05/28 08:43:18 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.243190 s / iter per device, on 1 devices)
[05/28 08:43:18 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/28 08:43:18 d2.evaluation.coco_evaluation]: Saving results to ./output\coco_instances_results.json
[05/28 08:43:18 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[05/28 08:43:18 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/28 08:43:18 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[05/28 08:43:18 d2.evaluation.